In [1]:
###-----------------
### Import Libraries
###-----------------
import sys


sys.path.append('../')
 

import os 
import numpy as np
import pandas as pd
import seaborn as sns
from torchsummary import summary
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, mean_squared_error


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Global variables
inpDir = '../input' # Input Stored here
outDir = '../output' # output Here
modelDir = './models'# to save Models
subDir = 'checkpoints' # sub dir by dataset
RANDOM_STATE = 24
np.random.RandomState(seed = RANDOM_STATE)



BATCH_SIZE = 64
NOISE = 0.2 
ALPHA = 0.001 
EPOCHS = 45
TEST_SIZE = 0.2

In [4]:
#set the plotting parameters
params = {
    'legend.fontsize': 'medium',
    'figure.figsize':(15,6),
    'axes.labelsize':'medium',
    'axes.titlesize':'medium',
    'xtick.labelsize': 'medium',
    'ytick.labelsize':'medium',
    #'text.usetex':True,
}
plt.rcParams.update(params)
CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid')


$$\text{
    Load Weather Data
}$$

In [6]:
file_loc = 'text_gen\shakespeare.txt'
file_path = os.path.join(inpDir,file_loc)

text = open(file_path, 'rb').read().decode(encoding='utf-8')

text[:100]

FileNotFoundError: [Errno 2] No such file or directory: '../input\\text_gen\\shakespeare.txt'

$$
\text {Character Based Modelling}
$$

In [ ]:
vocab = sorted(set(text))

len(vocab)

In [ ]:
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
char2idx

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])
text_as_int.shape

In [ ]:
seq_len = 100
example_per_epoch = len(text) / seq_len+1

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
    print(i.numpy(), end=' : ')
    print(idx2char[i.numpy()])


In [ ]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

for item in sequences.take(2):
    print(item)
    print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_intput_target(chunk):
    input_text = chunk[:-1]  # First hundred charater
    target_text = chunk[1:]  # Last character
    return input_text, target_text


dataset = sequences.map(split_intput_target)

for inp_epx, tar_epx in dataset.take(2):
    print(repr(''.join(idx2char[inp_epx.numpy()])))
    print(repr(''.join(idx2char[tar_epx.numpy()])))

In [ ]:
BUFFRE_SIZE = 10000

dataset = dataset.shuffle(BUFFRE_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE):
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape=(None,), batch_size=batch_size),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            kernel_initializer='glorot_uniform',
            recurrent_initializer='orthogonal',
        ),
        tf.keras.layers.Dense(vocab_size)
    ])


In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
model.summary()

In [ ]:
# Unit Testing, Check for models health withount trianing
for inp_ex, tar_ex in dataset.take(1):
    ex_pred = model(inp_ex)
    

In [ ]:
ex_pred.shape

In [ ]:
sample_indices = tf.random.categorical(ex_pred[0], num_samples=1)
print(sample_indices.shape)

In [ ]:
sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()
print(sample_indices)
print(repr(''.join(idx2char[sample_indices])))

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
model.compile(
    optimizer = 'adam',
    loss = loss_fn,
    metrics = [
        'accuracy'
    ]
)

In [ ]:
check_point_path = os.path.join(outDir, subDir)
check_point_prefix = os.path.join(check_point_path, "checkpoint-{epoch}.keras")
check_point_prefix

In [ ]:
check_point_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=check_point_prefix,
)

In [ ]:
history = model.fit(
    dataset,
    epochs =EPOCHS,
    callbacks = [check_point_callback],
    verbose = 1,
)

In [ ]:
loss_df = pd.DataFrame(history.history)
loss_df.head()

In [ ]:
# Min loss
loss_df.loc[loss_df['loss'] ==loss_df['loss'].min()]

In [ ]:
ax = loss_df.plot()
ax.vlines(
    loss_df.loc[loss_df['loss'] ==loss_df['loss'].min()].index,
    ymin=loss_df.loc[loss_df['loss'] ==loss_df['loss'].min()].values[0][1],
    ymax=loss_df.loc[loss_df['loss'] ==loss_df['loss'].max()].values[0][1],
) 

$$
\text{Best model}
$$

In [ ]:
# Best model
num = loss_df.loc[loss_df['loss'] ==loss_df['loss'].min()].index.to_numpy()[0]
# Fabricate full path
check_point_weight_path = os.path.join(check_point_path, f"checkpoint-{num}.keras")
check_point_weight_path

In [ ]:
# initintalize new model
model1 = build_model(
    vocab_size=vocab_size, 
    embedding_dim=embedding_dim, 
    rnn_units=rnn_units, 
    batch_size=1
)

model1.load_weights(check_point_weight_path)
model1.build(tf.TensorShape([1, None]))

model1.summary()

In [ ]:
def generate_text(model, start_sting):
    num_generate = 1000
    # Getting indices for start string
    input_eval = [char2idx[s] for s in start_sting] # List construct
    print(input_eval)
    input_eval = tf.expand_dims(input_eval, 0) # Size [1,1,5]
    print(input_eval)
    # store generated text
    generated_text = []
    for i in range(num_generate):
        prediction = model(input_eval)
        prediction = tf.squeeze(prediction,0)
        # this is where rummer meets the road
        prediction_td = tf.random.categorical(prediction, num_samples=1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([prediction_td],0)
        generated_text.append(idx2char[prediction_td])
    return start_sting + ''.join(generated_text)

In [ ]:
print(generate_text(model=model1, start_sting='ROMEO:'))